In [11]:
import csv as csv
import collections as col
import urllib.request as req
import json as js

enc_mdfile = '/home/pebert/work/code/mpggit/creepiest/datasrc/encode/20160211_ENCODE_metadata.tsv'

outfile = '/home/pebert/work/code/mpggit/creepiest/datasrc/encode/20160224_ENCODE_datasets.json'

encds = '/home/pebert/work/code/mpggit/creepiest/datasrc/encode/20160225_ENCDS_ids.json'

base_url = 'https://www.encodeproject.org/experiments/{}/?format=json'

def query_encode():
    dsmap = col.defaultdict(list)
    with open(enc_mdfile, 'r') as infile:
        rows = csv.DictReader(infile, delimiter='\t')
        for r in rows:
            expacc = r['Experiment accession']
            if expacc in dsmap:
                # TODO if ever needs to rerun:
                # add set of control exp ids and check if already seen
                print('Skip')
                continue
            query = base_url.format(expacc)
            print('Query ', query)
            resource = req.urlopen(query)
            obj = js.loads(resource.read().decode('utf-8'))
            assert obj['accession'] == expacc, 'That is unexpected... {} - {}'.format(expacc, obj['accession'])
            for entry in obj['possible_controls']:
                print('Controls found...')
                match_acc = entry['accession']
                dsmap[expacc].append(match_acc)
            resource.close()
            print('Done ', len(dsmap))
    print('Dumping JSON...')
    with open(outfile, 'w') as outf:
        js.dump(dsmap, outf)
    return


def assign_dataset_ids(infile, outfile):
    
    dsmap = js.load(open(infile))
    map_control_signal = col.defaultdict(set)
    for exp, controls in dsmap.items():
        for c in controls:
            map_control_signal[c].add(exp)
            
    map_dsid_expid = dict()
    for idx, control in enumerate(sorted(map_control_signal.keys()), start=1):
        dsid = 'ENCDS' + str(idx).zfill(3) + 'CRP'
        map_dsid_expid[control] = dsid
        for expid in map_control_signal[control]:
            map_dsid_expid[expid] = dsid
            
    print('ENCDS ids ', len(set(map_dsid_expid.values())))
    print('ENCSR ids ', len(map_dsid_expid))
    
    with open(outfile, 'w') as outf:
        js.dump(map_dsid_expid, outf, indent=1)
    return
    
# executed on 2016-02-24
# query_encode()

assign_dataset_ids(outfile, encds)

ENCDS ids  152
ENCSR ids  1026
